# Prediction Using KNN

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Importing the Necessary Libraries<br></p>

In [150]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import time

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Creating a Pandas DataFrame from a CSV file<br></p>

In [2]:
def CreateDataFrame(types, level):
    data = pd.read_csv('../Extraction/'+types+"/"+level+'.csv')
    data["Class"] = level
    return data

In [3]:
def LoadData(types):
    frames = [CreateDataFrame(types,"Clean"), 
          CreateDataFrame(types,"Dirty (0-20)"), 
          CreateDataFrame(types,"Dirty (20-40)"), 
          CreateDataFrame(types,"Dirty (40-60)"),
          CreateDataFrame(types,"Dirty (60-80)"),
          CreateDataFrame(types,"Dirty (80-100)")]
    df = pd.concat(frames)
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    return X,y

In [4]:
def LoadMinimizedData(types):
    df = pd.read_csv('../Feature Selection/FeatureSelection-'+types+".csv")
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    return X,y

# Classification using KNN

## Optimize Classfiers

In [174]:
def optimize(X_train, y_train):
    param_grid = [{'clf__weights': ["uniform", "distance"], 'clf__n_neighbors': [5,10]}]
    clfpipeline = Pipeline([
        ('scale', StandardScaler()),
        ('clf', KNeighborsClassifier())
    ])
    grid_search = GridSearchCV(clfpipeline, param_grid, cv=5, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    end = time.time()
    return grid_search

## Cycle Work

In [206]:
def cycleWork(X_train, X_test, y_train, y_test):
    gridsearch = optimize(X_train, y_train)
    traintime = gridsearch.cv_results_['mean_fit_time'].mean()*1000
    print("Mean Fit Time", traintime)
    print("Mean Score Time", gridsearch.cv_results_['mean_score_time'].mean()*1000)

    print(gridsearch.best_params_)
    estimator = gridsearch.best_estimator_
    
    y_train_pred = estimator.predict(X_train)
    trnscore = accuracy_score(y_train, y_train_pred)
    start = time.time()
    y_tst_pred = estimator.predict(X_test)
    end = time.time()
    diff = end-start
    clfperms = X_train.shape[0]/(diff*1000)
    tstscore = accuracy_score(y_test, y_tst_pred)
    cm = confusion_matrix(y_test, y_tst_pred)
    true_pos = np.diag(cm) 
    precision = np.sum(true_pos / np.sum(cm, axis=0))
    recall = np.sum(true_pos / np.sum(cm, axis=1))
    f1 = 2 * ((precision* recall)/(precision + recall))
    #f1None = f1_score(y_test, y_tst_pred, average=None)
    #f1micro = f1_score(y_test, y_tst_pred, average='micro')
    return [trnscore, tstscore, f1,traintime,clfperms]

In [207]:
def AllWork(types,cycles = 5):
    print(types)
    data = []
    for i in range(0,cycles):
        X,y = LoadData(types)
        #X,y = LoadMinimizedData(types)
        skf = StratifiedKFold(n_splits=5,shuffle = True)
        j = 0
        for train_index, test_index in skf.split(X, y.values.ravel()):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            acc = cycleWork(X_train, X_test, y_train, y_test)
            print("Train Accuracy ",acc[0],", Test Accuracy",acc[1], "F1 ",acc[2])
            data.append([i,j,acc[0],acc[1],acc[2],acc[3],acc[4]])
            j = j+1
    numpyarray = np.array(data)
    df = pd.DataFrame(numpyarray, columns = ["Cycle","Split","Train Acc", "Test Acc", "F1 Score","Train Time (ms)","clfs per MS"])
    df.to_csv("KNN-"+types+'.csv',index = False)
    df.head(10)

In [209]:
types = "NoBG"
AllWork(types,5)
#types = "Solar Data"
#AllWork(types,5)

NoBG
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    8.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    8.3s finished


Mean Fit Time 23.8610744476
Mean Score Time 87.1016979218
{'clf__n_neighbors': 5, 'clf__weights': 'distance'}
Train Accuracy  1.0 , Test Accuracy 0.967857142857 F1  5.80248187556
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    8.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    8.7s finished


Mean Fit Time 17.1406388283
Mean Score Time 68.4598445892
{'clf__n_neighbors': 5, 'clf__weights': 'distance'}
Train Accuracy  1.0 , Test Accuracy 0.960573476703 F1  5.7819457658
Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

Mean Fit Time 27.3337483406
Mean Score Time 108.838069439
{'clf__n_neighbors': 5, 'clf__weights': 'distance'}
Train Accuracy  1.0 , Test Accuracy 0.974910394265 F1  5.87809390706
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    8.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    8.7s finished


Mean Fit Time 19.7884559631
Mean Score Time 72.1086859703
{'clf__n_neighbors': 5, 'clf__weights': 'distance'}
Train Accuracy  1.0 , Test Accuracy 0.942238267148 F1  5.65947928126
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   10.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   10.5s finished


Mean Fit Time 19.1389203072
Mean Score Time 73.5089182854
{'clf__n_neighbors': 5, 'clf__weights': 'distance'}
Train Accuracy  1.0 , Test Accuracy 0.963898916968 F1  5.79210062997
